In [1]:
from model import *
import pytchat


C:\Users\avika\OneDrive\Documents\UAL\interactive_dance_thesis


In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="michellejieli/emotion_text_classifier")
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("michellejieli/emotion_text_classifier")
model = AutoModelForSequenceClassification.from_pretrained("michellejieli/emotion_text_classifier")

In [3]:
# Preparing MEED data for model
MEED= prep_MEED_data()
train_data,val_data,frame_dim,max_x,min_x,max_y,min_y,max_dx,min_dx,max_dy,min_dy = MEED

100%|██████████| 4102/4102 [00:03<00:00, 1054.20it/s]


Validating interpolation...
No errors found!


4102it [00:01, 2416.58it/s]
4102it [00:01, 2270.88it/s]


Creating keypoint frames...


100%|██████████| 4102/4102 [00:03<00:00, 1199.06it/s]


...

Validating length of dkp_frames
Validating length of kp_frames
Adding deltas to frames...


4102it [00:02, 1784.76it/s]


Validating length of data after delta
Adding emotions to frames...


100%|██████████| 4102/4102 [00:00<00:00, 34903.89it/s]


Validating length of data after emotions
frame_dim: 107


In [4]:
def get_video_by_emotion(data, specific_emotion):
    """
    Retrieve a single video of a specified emotion from the dataset.
    
    Parameters:
    - data (list): The entire dataset, assumed to be a list of videos.
    - specific_emotion (str): The emotion label of the desired video.
    
    Returns:
    - list: A single video corresponding to the specified emotion.
    """
    # Convert the specific emotion to its encoding
    specific_emotion_tuple = tuple(emotion_to_encoding(specific_emotion))
    
    # Search the dataset for a video with the specified emotion - return as tensor
    for video in data:
        if tuple(video[0][-7:]) == specific_emotion_tuple:
            return torch.tensor([video]).to(device).float()
    
    # If no video with the specified emotion is found, return None
    return None




# Example usage
specific_emotion_video = get_video_by_emotion(train_data, "Anger")

In [5]:
emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

In [23]:
chat = pytchat.create(video_id="jfKfPfyJRdk")

# Initialize a dictionary to keep track of cumulative scores and counts for each emotion
emotion_data = {emotion: {"score": 0.0, "count": 0} for emotion in emotion_labels}

# List to store average scores indexed by emotions
average_scores = [0.0 for _ in emotion_labels]
length =0


# create model
CHECKPOINT_PATH = "checkpoints/proto5_checkpoint.pth"

m = MotionModel(input_dim=frame_dim, output_dim=frame_dim, blocksize=BLOCK_SIZE, hidden_dim=512, n_layers=8, dropout=DROPOUT)
m = m.to(device)
optimizer = torch.optim.Adam(m.parameters(), lr=LEARNING_RATE, weight_decay=L2_REG)  # weight_decay=1e-5 L2 regularization
m, optimizer,scheduler, epoch, loss, train_seed = load_checkpoint(m, optimizer, CHECKPOINT_PATH)

# initial generation always neutral
generated = m.generate(specific_emotion_video, FRAMES_GENERATE)
unnorm_out = unnormalise_list_2D(generated, max_x, min_x, max_y, min_y,max_x, min_x, max_y, min_y)

# visualize
visualise_skeleton(unnorm_out[0], max_x, max_y, max_frames=FRAMES_GENERATE,save = False,save_path=None,prefix=f'adam_{EPOCHS}_coord_switchhappytodisgust',train_seed=train_seed,delta=False)


# have to normalise the generated data before putting it into the model

Loading checkpoint...
Checkpoint loaded from checkpoints/proto5_checkpoint.pth


Visualizing frames: 100%|██████████| 300/300 [00:32<00:00,  9.18it/s]


In [40]:
def normalise_generated(unnorm_out, max_x, min_x, max_y, min_y, max_dx, min_dx, max_dy, min_dy):
    norm_out = []
    
    for frame in unnorm_out:
        norm_frame = []
        
        # Normalize the first 50 values (absolute x and y coordinates)
        for i in range(0, 50, 2):
            unnormalized_x = frame[i]
            unnormalized_y = frame[i+1]
            
            norm_x = 2 * (unnormalized_x - min_x) / (max_x - min_x) - 1
            norm_y = 2 * (unnormalized_y - min_y) / (max_y - min_y) - 1
            
            norm_frame.extend([norm_x, norm_y])
        
        # Normalize the second 50 values (x and y deltas)
        for i in range(50, 100, 2):
            unnormalized_dx = frame[i]
            unnormalized_dy = frame[i+1]
            
            norm_dx = 2 * (unnormalized_dx - min_dx) / (max_dx - min_dx) - 1
            norm_dy = 2 * (unnormalized_dy - min_dy) / (max_dy - min_dy) - 1
            
            norm_frame.extend([norm_dx, norm_dy])
        
        # Append the emotion encoding without normalizing
        norm_frame.extend(frame[-7:])
        norm_out.append(norm_frame)
        
    return norm_out


In [27]:
np.shape(unnorm_out[0][-16:])

(16, 107)

In [41]:
average_scores= [0.0, 0.0, 0.0, 0.0, 0.9258161187171936, 0.0, 0.0]

# Generate a new batch based on the current average scores
last_frames = unnorm_out[0][-16:]
norm_last_frames = normalise_generated(last_frames, max_x, min_x, max_y, min_y,max_x, min_x, max_y, min_y)
new_batch = [frame[:-7] + average_scores for frame in norm_last_frames]
new_input = torch.tensor([new_batch]).to(device).float()

# Generate the new frames

generated = m.generate(new_input, FRAMES_GENERATE)
unnorm_out = unnormalise_list_2D(generated, max_x, min_x, max_y, min_y, max_x, min_x, max_y, min_y)

# visualize
visualise_skeleton(unnorm_out[0], max_x, max_y, max_frames=FRAMES_GENERATE,save = False,save_path=None,prefix=f'adam_{EPOCHS}_coord_switchhappytodisgust',train_seed=train_seed,delta=False)


Visualizing frames:   0%|          | 0/300 [00:00<?, ?it/s]

In [36]:
last_frames

[[280.3955993652344,
  161.3013916015625,
  264.75689697265625,
  213.18411254882812,
  237.95579528808594,
  202.04209899902344,
  224.8491668701172,
  253.27081298828125,
  232.78717041015625,
  318.73199462890625,
  289.9361877441406,
  225.2308349609375,
  283.19696044921875,
  278.7978515625,
  267.3387145996094,
  304.4544677734375,
  249.85452270507812,
  317.9267272949219,
  231.56967163085938,
  314.1739501953125,
  197.7384033203125,
  402.1186218261719,
  162.34036254882812,
  511.0916748046875,
  268.33587646484375,
  320.4405212402344,
  266.8221740722656,
  396.05401611328125,
  272.53448486328125,
  461.7596130371094,
  276.32464599609375,
  157.70571899414062,
  286.06365966796875,
  160.31663513183594,
  265.3162536621094,
  172.26951599121094,
  296.50360107421875,
  183.28421020507812,
  307.0704040527344,
  475.0675048828125,
  304.1018371582031,
  468.4628601074219,
  264.5594482421875,
  468.3992004394531,
  160.3931884765625,
  530.2598876953125,
  153.8160400390

In [39]:
unnormalise_list_2D(new_input,max_x, min_x, max_y, min_y, max_x, min_x, max_y, min_y)

[[[475.5338134765625,
   347.9902038574219,
   467.71441650390625,
   373.9315185546875,
   454.31390380859375,
   368.36053466796875,
   447.7605895996094,
   393.97491455078125,
   451.7295837402344,
   426.7054748535156,
   480.3040771484375,
   379.95489501953125,
   476.9344787597656,
   406.7384033203125,
   469.0053405761719,
   419.5667419433594,
   460.2632751464844,
   426.3028564453125,
   451.120849609375,
   424.42645263671875,
   434.2052001953125,
   468.3988037109375,
   416.5061950683594,
   522.8853149414062,
   469.50396728515625,
   427.55975341796875,
   468.7471008300781,
   465.3664855957031,
   471.603271484375,
   498.21929931640625,
   473.4983215332031,
   346.1923522949219,
   478.3677978515625,
   347.497802734375,
   467.994140625,
   353.4742431640625,
   483.5877990722656,
   358.9815979003906,
   488.8712158203125,
   504.87322998046875,
   487.3869323730469,
   501.5709228515625,
   467.61572265625,
   501.5390625,
   415.5325927734375,
   532.46942138

In [35]:
norm_last_frames

[[0.3797337880933065,
  0.2731299135326301,
  0.3548792012507412,
  0.37413221525838947,
  0.31228421526189665,
  0.352441583120308,
  0.29145385312250083,
  0.45217070165718654,
  0.30406972069193067,
  0.5796067709048153,
  0.39489664345304726,
  0.3975840826558426,
  0.384185990749987,
  0.5018652744391143,
  0.3589824835314518,
  0.5518121015230876,
  0.3311948605621101,
  0.5780391227022537,
  0.30213475010542523,
  0.5707334317515126,
  0.2483667215880673,
  0.7419390681764276,
  0.19210862188606997,
  0.9540815498342495,
  0.3605672727008732,
  0.5829328323979904,
  0.35816154576613807,
  0.7301328420402977,
  0.3672401202579962,
  0.8580447249254872,
  0.3732638226508182,
  0.2661300645854054,
  0.3887420352922599,
  0.27121284618793917,
  0.3557681867346379,
  0.29448202823946906,
  0.40533423225331205,
  0.3159248024057808,
  0.42212805028788475,
  0.8839517645524118,
  0.4174101074671871,
  0.8710942213816614,
  0.3545653960196423,
  0.8709702923957765,
  0.18901397849035667

In [34]:
unnorm_out[0]

[[475.5338134765625,
  347.9902038574219,
  467.71441650390625,
  373.9315185546875,
  454.31390380859375,
  368.36053466796875,
  447.7605895996094,
  393.97491455078125,
  451.7295837402344,
  426.7054748535156,
  480.3040771484375,
  379.95489501953125,
  476.9344787597656,
  406.7384033203125,
  469.0053405761719,
  419.5667419433594,
  460.2632751464844,
  426.3028564453125,
  451.120849609375,
  424.42645263671875,
  434.2052001953125,
  468.3988037109375,
  416.5061950683594,
  522.8853149414062,
  469.50396728515625,
  427.55975341796875,
  468.7471008300781,
  465.3664855957031,
  471.603271484375,
  498.21929931640625,
  473.4983215332031,
  346.1923522949219,
  478.3677978515625,
  347.497802734375,
  467.994140625,
  353.4742431640625,
  483.5877990722656,
  358.9815979003906,
  488.8712158203125,
  504.87322998046875,
  487.3869323730469,
  501.5709228515625,
  467.61572265625,
  501.5390625,
  415.5325927734375,
  532.4694213867188,
  412.2440490722656,
  531.899597167968

In [8]:
import threading
import queue

In [ ]:


def generate_new_batch():
    # prep new input from chat emotion average scores and last 16 frames of generated
    last_frames = unnorm_out[0][-16:]
    new_batch = [frame[:-7] + average_scores for frame in unnorm_out[0][-16:]]
    new_batch = torch.tensor(new_batch).to(device).float()

    # generate new frames
    generated = m.generate(new_batch, FRAMES_GENERATE)
    return unnormalise_list_2D(generated, max_x, min_x, max_y, min_y, max_x, min_x, max_y, min_y)

def visualize(unnorm_out):
    # visualize
    visualise_skeleton(unnorm_out[0], max_x, max_y, max_frames=FRAMES_GENERATE, save=False, save_path=None, prefix=f'adam_{EPOCHS}_coord_switchhappytodisgust', train_seed=train_seed, delta=False)


# This queue will hold the batches ready for visualization
viz_queue = queue.Queue()

def visualize_batches():
    while True:
        # Wait for a new batch to be available in the queue
        unnorm_out = viz_queue.get()
        
        # Visualize the batch
        visualize(unnorm_out)

# Start the visualization thread
threading.Thread(target=visualize_batches, daemon=True).start()

while chat.is_alive():
    for c in chat.get().sync_items():
        print(f"{c.datetime} [{c.author.name}]- {c.message}")
        result = pipe(c.message)  # Assuming pipe() returns emotion prediction
        print(result)
        
        # Update emotion data with new score and count
        emotion = result[0]['label']
        score = result[0]['score']
        emotion_data[emotion]["score"] += score
        emotion_data[emotion]["count"] += 1
        
        # Compute average scores for each emotion
        for i, emotion in enumerate(emotion_labels):
            if emotion_data[emotion]["count"] > 0:
                average_scores[i] = emotion_data[emotion]["score"] / emotion_data[emotion]["count"]
        
        # Print average scores (can also store them elsewhere if needed)
        print("Average scores:", average_scores)
        length  +=1
        
        if length % 10 == 0:
            # Generate a new batch based on the current average scores
            last_frames = unnorm_out[0][-16:]
            new_batch = [frame[:-7] + average_scores for frame in last_frames]
            new_batch = torch.tensor([new_batch]).to(device).float()

            # Generate the new frames
            generated = m.generate(new_batch, FRAMES_GENERATE)
            unnorm_out = unnormalise_list_2D(generated, max_x, min_x, max_y, min_y, max_x, min_x, max_y, min_y)

            # Put the batch in the visualization queue
            viz_queue.put(unnorm_out)



2023-10-19 19:22:34 [ro]- اوبااااا يا رؤى
[{'label': 'neutral', 'score': 0.9258161187171936}]
Average scores: [0.0, 0.0, 0.0, 0.0, 0.9258161187171936, 0.0, 0.0]
2023-10-19 19:22:36 [Xa]- Elina الحفظ حلو همين
[{'label': 'neutral', 'score': 0.8884650468826294}]
Average scores: [0.0, 0.0, 0.0, 0.0, 0.9071405827999115, 0.0, 0.0]
2023-10-19 19:22:38 [Baka ハンサム 🇵🇸]- zara thanks :folded_hands:
[{'label': 'joy', 'score': 0.6992315053939819}]
Average scores: [0.0, 0.0, 0.0, 0.6992315053939819, 0.9071405827999115, 0.0, 0.0]
2023-10-19 19:22:39 [Rhishabh Suneeth]- how are yall doin
[{'label': 'neutral', 'score': 0.9851313233375549}]
Average scores: [0.0, 0.0, 0.0, 0.6992315053939819, 0.9331374963124593, 0.0, 0.0]
2023-10-19 19:22:39 [zoro]- TOPPER:backhand_index_pointing_down:
[{'label': 'neutral', 'score': 0.9879993200302124}]
Average scores: [0.0, 0.0, 0.0, 0.6992315053939819, 0.9468529522418976, 0.0, 0.0]
2023-10-19 19:22:41 [Suzan Demir]- benim bir gidip gelicem
[{'label': 'neutral', 'score':

Visualizing frames:   3%|▎         | 9/300 [00:00<00:31,  9.13it/s]

2023-10-19 19:22:50 [𝘻𝘢𝘳𝘢 ༒]- حرامات الوقت عدهم معكوس من اقرا بالليل هي عدها صبح
[{'label': 'neutral', 'score': 0.8088387250900269}]
Average scores: [0.0, 0.0, 0.0, 0.6992315053939819, 0.8732951939105987, 0.0, 0.0]


Visualizing frames:   6%|▌         | 18/300 [00:01<00:30,  9.10it/s]

2023-10-19 19:22:50 [doneswar]- ananya but why you scolding me :clown_face:
[{'label': 'anger', 'score': 0.7776647210121155}]
Average scores: [0.7776647210121155, 0.0, 0.0, 0.6992315053939819, 0.8732951939105987, 0.0, 0.0]


...

Visualizing frames:   8%|▊         | 23/300 [00:02<00:29,  9.26it/s]

2023-10-19 19:24:38 [mys]- wo din bhi kya din the
[{'label': 'neutral', 'score': 0.9692026376724243}]
Average scores: [0.7689014256000519, 0.0, 0.4819953143596649, 0.7741999093975339, 0.8790328788991068, 0.6856129964192709, 0.4198499023914337]


Visualizing frames:  13%|█▎        | 40/300 [00:05<01:14,  3.48it/s]

KeyboardInterrupt: 

Visualizing frames:  26%|██▌       | 77/300 [00:08<00:24,  9.14it/s]

In [7]:

while chat.is_alive():
    for c in chat.get().sync_items():
        print(f"{c.datetime} [{c.author.name}]- {c.message}")
        result = pipe(c.message)  # Assuming pipe() returns emotion prediction
        print(result)
        
        # Update emotion data with new score and count
        emotion = result[0]['label']
        score = result[0]['score']
        emotion_data[emotion]["score"] += score
        emotion_data[emotion]["count"] += 1
        
        # Compute average scores for each emotion
        for i, emotion in enumerate(emotion_labels):
            if emotion_data[emotion]["count"] > 0:
                average_scores[i] = emotion_data[emotion]["score"] / emotion_data[emotion]["count"]
        
        # Print average scores (can also store them elsewhere if needed)
        print("Average scores:", average_scores)
        length  +=1
        
        if length % 10 ==0 :
            generated = m.generate(xb, FRAMES_GENERATE)


2023-10-19 18:55:18 [ Ar. Sharma~]- :cigarette:
[{'label': 'neutral', 'score': 0.9502205848693848}]
Average scores: [0.0, 0.0, 0.0, 0.0, 0.9502205848693848, 0.0, 0.0]
2023-10-19 18:55:18 [Michael Terry Wilson]- Best wishes and love to everyone.
[{'label': 'neutral', 'score': 0.682776689529419}]
Average scores: [0.0, 0.0, 0.0, 0.0, 0.8164986371994019, 0.0, 0.0]


KeyboardInterrupt: 

In [ ]:
# Preparing MEED data for model
MEED= prep_MEED_data()
train_data,val_data,frame_dim,max_x,min_x,max_y,min_y,max_dx,min_dx,max_dy,min_dy = MEED

In [2]:


CHECKPOINT_PATH = "checkpoints/proto5_checkpoint.pth"

# create model
m = MotionModel(input_dim=frame_dim, output_dim=frame_dim, blocksize=BLOCK_SIZE, hidden_dim=512, n_layers=8, dropout=DROPOUT)
m = m.to(device)
optimizer = torch.optim.Adam(m.parameters(), lr=LEARNING_RATE, weight_decay=L2_REG)  # weight_decay=1e-5 L2 regularization
m, optimizer,scheduler, epoch, loss, train_seed = load_checkpoint(m, optimizer, CHECKPOINT_PATH)
# Generate a sequence
print(f'Generating sequence of {FRAMES_GENERATE} frames...')

generated = m.generate(, FRAMES_GENERATE)
unnorm_out = unnormalise_list_2D(generated, max_x, min_x, max_y, min_y,max_x, min_x, max_y, min_y)



100%|██████████| 4102/4102 [00:03<00:00, 1086.69it/s]


Validating interpolation...
No errors found!


4102it [00:01, 2499.12it/s]
4102it [00:01, 2397.84it/s]


Creating keypoint frames...


100%|██████████| 4102/4102 [00:03<00:00, 1039.36it/s]


...

4102it [00:03, 1110.06it/s]


Validating length of data after delta
Adding emotions to frames...


100%|██████████| 4102/4102 [00:00<00:00, 9863.41it/s]


Validating length of data after emotions
frame_dim: 107
Loading checkpoint...
Checkpoint loaded from checkpoints/proto5_checkpoint.pth
Generating sequence of 300 frames...


NameError: name 'batch' is not defined

In [19]:
def encoding_to_emotions(encoding):
    """
    Convert an encoding to its corresponding emotion labels.
    
    Parameters:
    - encoding (list of int): The encoding of the emotions.
    
    Returns:
    - list of str: The labels of the emotions.
    """
    
    emotion_labels = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sad', 'Surprise']
    
    # Find the indices of non-zero values in the encoding and return the corresponding labels
    emotion_indices = [i for i, value in enumerate(encoding) if value > 0]
    return [emotion_labels[i] for i in emotion_indices]

# Test the function with an example encoding
test_encoding = [0.97, 0, 1, 1, 0, 0, 0]
encoding_to_emotions(test_encoding)


['Anger', 'Fear', 'Happiness']

In [12]:
frame[-7:]

[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]

In [33]:
# switch emotions of frames 
def switch_emotion(frame,old_emotion,new_emotion):
    """
    Switch the emotion of a frame.
    
    Parameters:
    - frame (list of int): The frame to switch the emotion of.
    - old_emotion (str): The emotion to switch from.
    - new_emotion (str): The emotion to switch to.
    
    Returns:
    - list of int: The frame with the emotion switched.
    """
    
    # Get the indices of the old and new emotion
    emotion_labels = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sad', 'Surprise']
    old_emotion_index = emotion_labels.index(old_emotion)
    new_emotion_index = emotion_labels.index(new_emotion)
    
    # Switch the emotion
    frame[-7 + old_emotion_index] = 0
    frame[-7 + new_emotion_index] = 1
    
    return frame

new_batch = []
for frame in batch:
    emotion = encoding_to_emotions(frame[-7:])
    print(emotion)
    new_batch.append(frame[:-7] + emotion_to_encoding('Disgust'))

['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Anger', 'Disgust', 'Happiness', 'Neutral']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Happiness', 'Neutral', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Sad', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Fear', 'Happiness']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Sur

In [24]:
for batch in unnorm_out:
    visualise_skeleton(batch, max_x, max_y, max_frames=FRAMES_GENERATE,save =True,save_path=None,prefix=f'adam_{EPOCHS}_coord_normalemotion_happy',train_seed=train_seed,delta=False)

Visualizing frames: 100%|██████████| 300/300 [00:33<00:00,  9.07it/s]


In [25]:
visualise_skeleton(new_batch, max_x, max_y, max_frames=FRAMES_GENERATE,save = True,save_path=None,prefix=f'adam_{EPOCHS}_coord_switchhappytosad',train_seed=train_seed,delta=False)

Visualizing frames:  27%|██▋       | 82/300 [00:09<00:24,  9.06it/s]


KeyboardInterrupt: 

In [34]:
happy_to_sad_xb = [new_batch[-16:]]
happy_to_sad_xb = torch.tensor(happy_to_sad_xb).to(device).float()


generated = m.generate(happy_to_sad_xb, FRAMES_GENERATE)
unnorm_out = unnormalise_list_2D(generated, max_x, min_x, max_y, min_y,max_x, min_x, max_y, min_y)


In [35]:
for new_batch in unnorm_out:

    visualise_skeleton(new_batch, max_x, max_y, max_frames=FRAMES_GENERATE,save = True,save_path=None,prefix=f'adam_{EPOCHS}_coord_switchhappytodisgust',train_seed=train_seed,delta=False)

Visualizing frames: 100%|██████████| 300/300 [00:32<00:00,  9.17it/s]


In [36]:
xb.shape

torch.Size([1, 16, 107])